In [10]:
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import *

In [11]:
#Import the dicyionnaries
import csv
from datetime import datetime

Nov5 = dict() 


# Chargement des donnees
with open("5Nov.csv") as Nov5_file:
    reader = csv.DictReader(Nov5_file) 

    for row in reader:
        if row['Numéro du groupe'] != '' :
            heures, minutes, secondes = map(int, row['TransitTime'].split(':'))
            total_minutes = heures * 60 + minutes 
            Nov5[int(float(row['Numéro du groupe']))] = (int(float(row['Femmes'])),int(float(row['Hommes'])),int(float(row['WCHR'])),total_minutes)  
            
print(Nov5) 

Passengers = {}
j = 1

for group_num, data in Nov5.items():
    weights = {'Femmes': 70, 'Hommes': 85, 'WCHR': 100}  # Weight per passenger
    
    # Extracting data
    femmes, hommes, wchr, total_minutes = data
    
    # Enumerating passengers
    for _ in range(femmes + hommes + wchr):
        if femmes > 0:
            weight = weights['Femmes']  
            Passengers[j] = {'gender': 'Femme', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            femmes -= 1
        elif hommes > 0:
            weight = weights['Hommes']
            Passengers[j] = {'gender': 'Homme', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            hommes -= 1
        else:
            weight = weights['WCHR']
            Passengers[j] = {'gender': 'WCHR', 'group': group_num, 'weight': weight, 'connection_time': total_minutes}
            wchr -= 1
        j += 1

print(Passengers)



{1: (1, 2, 0, 0), 2: (0, 1, 0, 60), 3: (1, 1, 0, 165), 4: (1, 1, 1, 0), 5: (3, 0, 0, 0), 6: (1, 1, 0, 0), 7: (1, 1, 0, 85), 8: (0, 1, 0, 0), 9: (2, 2, 0, 60), 10: (1, 0, 0, 0), 11: (1, 0, 0, 0), 12: (0, 2, 0, 0), 13: (0, 1, 0, 0), 14: (0, 3, 0, 0), 15: (2, 0, 0, 0), 16: (1, 0, 0, 60), 17: (1, 0, 0, 0), 18: (1, 1, 0, 0), 19: (0, 1, 0, 0), 20: (1, 1, 0, 0), 21: (0, 1, 0, 0), 22: (0, 1, 0, 0), 23: (2, 1, 0, 0), 24: (1, 1, 0, 0), 25: (0, 0, 0, 60), 26: (2, 0, 0, 0), 27: (0, 1, 0, 0), 28: (0, 1, 0, 0), 29: (0, 1, 0, 85), 30: (1, 1, 0, 85), 31: (2, 2, 0, 0), 32: (0, 5, 0, 0), 33: (0, 4, 0, 0), 34: (1, 1, 0, 85), 35: (1, 1, 0, 0), 36: (2, 0, 0, 60), 37: (0, 1, 0, 0), 38: (0, 1, 0, 0), 39: (1, 0, 1, 0), 40: (1, 0, 1, 0), 41: (0, 1, 0, 0), 42: (0, 1, 0, 165), 43: (5, 0, 0, 0), 44: (2, 0, 0, 85), 45: (0, 1, 0, 85), 46: (1, 2, 0, 0), 47: (1, 0, 0, 0), 48: (1, 0, 0, 0), 49: (0, 0, 1, 0), 50: (1, 0, 0, 0), 51: (0, 1, 0, 0), 52: (1, 0, 0, 0), 53: (1, 0, 0, 0)}
{1: {'gender': 'Femme', 'group': 1, 'we

Passengers : dictionnaire extrait du fichier excel

Les sièges sont comptés comme une liste et pas comme une matrice

Sij : var. binaire qui à un passager i associe le siège j

In [12]:
ranks = 29 #nombre de rangées dans l'avion
n = 90 #nombre de passagers  A MODIFIER
ns=7*ranks

def modele_statique():
    m=Model('statique')
    S = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'j{i}') for i in range(1, n+1) for j in range (1, ns+1)}

    #Au max 1 siège par passager
    for j in range (1, ns+1):
        m.addConstr(sum(S[(i,j)] for i in range (1, n+1)) <=1, name="PassengerMax")

    #Chaque passager a un et un seul siège
    for i in range (1, n+1):
        m.addConstr(sum(S[(i,j)] for j in range (1, ns+1)) == 1, name="SeatMax")


    #Barycentre

    return m, S

In [13]:
## Fonctions objectifs

m,S=modele_statique()

#Les passagers en transit sont placés à l'avant de l'avion

def obj_transit():
    T= {} 
    
    
    for passenger in Passengers:
        if Passengers[passenger]['connection_time'] >0:   #passager en transit
            T[passenger]=Passengers[passenger]['connection_time']

    #T=sorted(T.items(), key=lambda item:item[1], reverse=True)
    P=list(T.keys())  #retourne la liste des identifiants des passagers en transit
            

    f=0
    for k in range (len(P)):
        for j in range (1, ns+1):
            q=(j-1)//7
            f+= S[(P[k],j)] * (1/T[P[k]]) * q
            

    return f

m.setObjective(obj_transit(), GRB.MINIMIZE)



In [14]:
m.params.outputflag = 0
#m.display()
m.update()
m.optimize()
print("Les places dans l'avion sont les suivantes :", [S[(i,j)].x for i in range(1,n+1) for j in range (1,ns+1)])

Les places dans l'avion sont les suivantes : [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 